In [1]:
from pruning import *
from lw_retrain_utils import *
from evaluation import *
import json
import copy 
from datasets import load_dataset
import os
import gc
import shutil

/users/eleves-a/2021/ahmad.rammal/miniconda3/envs/colab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "openai-community/gpt2-medium"
model, tokenizer = load_model(model_name)

In [3]:
prune_model_depth(model, tokenizer, 4)

Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
100%|█████████▉| 280/281 [00:06<00:00, 44.16it/s]


Perplexity on Wikitext-2: 21.97
Initial PPL: 21.97


Evaluating iterations...:   0%|          | 0/4 [00:00<?, ?it/s]


























































































100%|█████████▉| 280/281 [00:13<00:00, 21.16it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 45.46it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 44.37it/s]






























































































100%|█████████▉| 280/281 [00:13<00:00, 20.31it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 46.37it/s]

























































100%|█████████▉| 280/281 [00:06<00:00, 45.24it/s]



























































































100%|█████████▉| 280/281 [00:13<00:00, 20.96it/s]


























































100%|█████████▉| 280/281 

Removed layer 2. 23 layers left. PPL: 22.45
New model size:  342226944




























































































100%|█████████▉| 280/281 [00:12<00:00, 22.35it/s]

























































100%|█████████▉| 280/281 [00:05<00:00, 47.80it/s]































































100%|█████████▉| 280/281 [00:07<00:00, 36.01it/s]






















































































100%|█████████▉| 280/281 [00:11<00:00, 23.61it/s]

























































100%|█████████▉| 280/281 [00:05<00:00, 47.77it/s]





































































100%|█████████▉| 280/281 [00:08<00:00, 31.22it/s]
















































































100%|█████████▉| 280/281 [00:10<00:00, 26.34it/s]

























































100%|█████████▉| 280/281 [00:05<00:00, 47.77it/s]











































Removed layer 9. 22 layers left. PPL: 23.50
New model size:  329630720

























































100%|█████████▉| 280/281 [00:05<00:00, 47.92it/s]



























































































100%|█████████▉| 280/281 [00:12<00:00, 22.45it/s]





























































100%|█████████▉| 280/281 [00:06<00:00, 43.59it/s]
























































100%|█████████▉| 280/281 [00:05<00:00, 46.92it/s]






























































































100%|█████████▉| 280/281 [00:12<00:00, 21.92it/s]

























































100%|█████████▉| 280/281 [00:05<00:00, 48.74it/s]
























































100%|█████████▉| 280/281 [00:05<00:00, 48.92it/s]


























































































100%|█████████▉| 280/281 [00:12<00:00, 22.86it/s]










































Removed layer 19. 21 layers left. PPL: 24.69
New model size:  317034496




















































100%|█████████▉| 280/281 [00:05<00:00, 48.90it/s]






























































































100%|█████████▉| 280/281 [00:12<00:00, 22.28it/s]


















































100%|█████████▉| 280/281 [00:05<00:00, 48.52it/s]















































100%|█████████▉| 280/281 [00:05<00:00, 51.39it/s]






































































100%|█████████▉| 280/281 [00:08<00:00, 31.98it/s]













































































100%|█████████▉| 280/281 [00:09<00:00, 28.07it/s]















































100%|█████████▉| 280/281 [00:05<00:00, 51.50it/s]





























































100%|█████████▉| 280/281 [00:07<00:00, 37.59it/s]






















































































100%|█████████▉| 2

Removed layer 11. 20 layers left. PPL: 26.47
New model size:  304438272


In [4]:
dataset = load_dataset("stas/openwebtext-10k", trust_remote_code=True)
calibration_pass(model=model,
                 tokenizer=tokenizer,
                 dataset=dataset,
                 sample_size=128,
                 batch_size=4,)

100%|██████████| 32/32 [00:13<00:00,  2.44it/s]


In [8]:
tokenized_dataset = tokenize_dataset(tokenizer, dataset)

os.makedirs("./saved_metrics", exist_ok=True)

training_metrics_path = "./saved_metrics/training_metrics.json"
eval_metrics_path = "./saved_metrics/eval_metrics.json"

if os.path.exists(training_metrics_path):
    with open(training_metrics_path, "r") as f:
        training_metrics = json.load(f)
else:
    training_metrics = {}

if os.path.exists(eval_metrics_path):
    with open(eval_metrics_path, "r") as f:
        eval_metrics = json.load(f)
else:
    eval_metrics = {}

num_heads = 12
mult_hidden = 3.5
embed_size = 1024
prune_model_width(model, int(mult_hidden * embed_size), num_heads, embed_size)
print(sum(t.numel() for t in model.parameters()))
param_key = f"depth=20_num_heads={num_heads}_mlp_exp={mult_hidden}_embed_size={embed_size}"
gc.collect()

if param_key+"_before_training" in eval_metrics:
    print(f"Skipping evaluation for {param_key}, already exists.")
else:
    print(f"Evaluating perplexity for {param_key} before training...")
    eval_metrics[param_key+"_before_training"] = evaluate_perplexity(model, tokenizer, stride=1024).item()

    with open(eval_metrics_path, "w") as f:
        json.dump(eval_metrics, f, indent=4)
if param_key in training_metrics:
    print(f"Skipping training for {param_key}, already exists.")
else:
    print(f"Training model for {param_key}...")

    
    trainer = trainer_gpt2(model, tokenizer, tokenized_dataset, batch_size=4, num_epochs=2, lr=2e-4, output_dir=f"./saved_models/{param_key}")
    trainer.train()
    training_metrics[param_key] = trainer.state.log_history

    with open(training_metrics_path, "w") as f:
        json.dump(training_metrics, f, indent=4)
    torch.cuda.empty_cache()
    gc.collect()
    
if param_key in eval_metrics:
    print(f"Skipping evaluation for {param_key}, already exists.")
else:
    print(f"Evaluating perplexity for {param_key}...")
    eval_metrics[param_key] = evaluate_perplexity(model, tokenizer, stride=1024).item()

    with open(eval_metrics_path, "w") as f:
        json.dump(eval_metrics, f, indent=4)

torch.cuda.empty_cache()
gc.collect()

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map: 100%|██████████| 9000/9000 [00:04<00:00, 2048.49 examples/s]


262469632
Evaluating perplexity for depth=20_num_heads=12_mlp_exp=3.5_embed_size=1024 before training...


100%|█████████▉| 280/281 [00:04<00:00, 57.96it/s]


Perplexity on Wikitext-2: 85.16
Training model for depth=20_num_heads=12_mlp_exp=3.5_embed_size=1024...


Step,Training Loss,Validation Loss
30,3.484400,3.121555
60,3.262300,3.037397
90,3.213700,3.007232
120,3.195500,2.989893
150,3.147300,2.985213
180,3.136500,2.980379
210,3.144700,2.979733
240,3.117700,2.979151
270,3.125000,2.979301


Evaluating perplexity for depth=20_num_heads=12_mlp_exp=3.5_embed_size=1024...


100%|█████████▉| 280/281 [00:06<00:00, 41.41it/s]


Perplexity on Wikitext-2: 36.63


33